In [6]:
import re
import time
import pandas as pd
import datetime as dt
import chromedriver_autoinstaller
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from tqdm.notebook import tqdm

In [7]:
# 실사용 할 무신사 URL
musinsaUrl = "https://www.musinsa.com/ranking/best?period=month&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page={}&viewType=small&priceMin=&priceMax="
pageNumber = range(1,2)

In [8]:
# 한글 제거 함수
def remove_korean(text):
    pattern = re.compile('[가-힣]+')  # 한글 패턴
    result = pattern.sub('', text)
    return result

In [9]:
musinsaData = []

chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(3)

for p in pageNumber:
    url = musinsaUrl.format(p)
    # print(url)
    
    driver.get(url)
    driver.implicitly_wait(3)

    soup1 = BeautifulSoup(driver.page_source)

    itemList = soup1.select(".li_box")

    for li in itemList:
        itemNo = li['data-goods-no']
        # print(itemNo)
        
        itemUrl = f'https://www.musinsa.com/app/goods/{itemNo}?loc=goods_rank'
        
        driver.refresh
        driver.get(itemUrl)
        driver.implicitly_wait(3)
        time.sleep(np.random.uniform(2,4))
        
        
        # 현재 페이지 소스를 가져와서 BeautifulSoup으로 파싱
        soup2 = BeautifulSoup(driver.page_source)
        
        # for elem in soup2.select('.product_title .product_title_eng, .product_title .txt_reserve, .product_title .txt_delay'):
        #   elem.decompose()
        
        
        # 상품명
        title = soup2.select('.product-detail__sc-1klhlce-3.fjguJZ')

        if title:
          titleValue = title[0].text.strip()
          titleValue = titleValue.replace('[무료반품]', '').strip()
        else:
          titleValue = ""
          
        # print(titleValue)
      
      
        #종류만 추출
        category = soup2.select('#product-top > div > div.product-detail__sc-up77yl-0.htaeEr > a:nth-child(1)')
        
        if category:
          categoryValue = category[0].text.strip()
          categoryValue = re.sub(r'\([^()]*\)', '', categoryValue)
          categoryValue = categoryValue.replace('>', '').strip()
        else:
          categoryValue = "" 
           
        # print(categoryValue)
        
        
        # 분류만 추출
        types = soup2.select('#product-top > div > div.product-detail__sc-up77yl-0.htaeEr > a:nth-child(3)')
        
        if types:
          typesValue = types[0].text.strip()
          typesValue = re.sub(r'\([^()]*\)', '', typesValue)
          typesValue = typesValue.replace('>', '').strip()
        else:
          typesValue = ""  
          
        # print(typesValue)
        
        
        # 착용성별 추출
        gender = soup2.select('#product-right-top > div.product-detail__sc-achptn-0.eXRtIE > ul > li:nth-child(2) > div.product-detail__sc-achptn-6.gfoaTb > span:nth-child(3)')

        if gender:
            genderText = [g.get_text(strip=True) for g in gender]
            genderValue = ' '.join(genderText)
            genderValue = genderValue.replace('남성, 여성','공용').strip()
        else:
            genderValue = ""
        # print(genderValue)
        
        
        # 조회수
        try:
          views = soup2.select('#product-right-top > div.product-detail__sc-achptn-0.eXRtIE > ul > li:nth-child(3) > div.product-detail__sc-achptn-6.gfoaTb > span')
          if views:
            viewText = [v.get_text(strip=True) for v in views]
            viewsValue = ' '.join(viewText)
            a = ''
            if "만" in viewsValue: a ='만'
            elif "천" in viewsValue: a ='천'

            viewsValue = float(remove_korean(viewsValue)) if viewsValue else 0

            if a:
                if a == '만':
                    viewsValue *= 10000
                else:
                    viewsValue *= 1000
                viewsValue = int(viewsValue)
          else:
              viewsValue = 0
        except:
          
            viewsValue = 0
            pass
          
        # print(viewsValue)
        
        
        # 누적판매
        try:
          sales = soup2.select('#product-right-top > div.product-detail__sc-achptn-0.eXRtIE > ul > li:nth-child(4) > div.product-detail__sc-achptn-6.gfoaTb > span')
        
          if sales:
              salesText = [s.get_text(strip=True) for s in sales]
              salesValue = ' '.join(salesText)
              a = ''
              if "만" in salesValue: a ='만'
              elif "천" in salesValue: a ='천'

              salesValue = float(remove_korean(salesValue)) if salesValue else 0

              if a:
                  if a == '만':
                      salesValue *= 10000
                  else:
                      salesValue *= 1000
                  salesValue = int(salesValue)
          else:
              salesValue = 0
        except:
          
            salesValue = 0
            pass
        
        # print(salesValue)
        
        
        # 선호연령대
        try:
          ages = soup2.select('#product-right-top > button > strong')

          if ages:
              agesText = [a.get_text(strip=True) for a in ages]
              agesValueList = []

              for ageText in agesText:
                  # ~을 기준으로 문자열을 분할하여 숫자를 추출
                  age_values = ageText.split('~')
                  agesValueList.extend(age_values)

              agesValue = ' '.join(agesValueList)
          else:
              agesValue = 0
        except:
          
            agesValue = 0
            pass
        
        # print(agesValue)
        
        
        # 연령대 선호 비율
        try:  
          ageratios = soup2.select('#page_product_detail > div.right_area.page_detail_product > div.section_graph_detail > div > div > div.graph_bar_wrap > div > ul')
        
          ageratiosValues = []
          for ageratio in ageratios:
              ageratioText = ageratio.text.strip()
              ageratioText = ageratioText.replace('%',',').replace('\n', '').replace(' ', '').strip()
              ageratiosValues.append(ageratioText)
        except:
          ageratiosValues = ''
          pass
        # print(ageratiosValues)       
        
        
        # 선호성별
        try:
          sex = soup2.select('#product-right-top > button > em')
          if sex:
            sexText = [x.get_text(strip=True) for x in sex]
            sexValue = '  '.join(sexText)
          else:
            sexValue = ""
        except:
          sexValue = ""
          pass
        # print(sexValue)
        
        
        # 남자 선호 비율
        try:
            maleRatio = soup2.select(".doughnut_label_wrap .label_info_value")[0]
            maleRatioValue = maleRatio.text.strip()
            maleRatioValue = maleRatioValue.replace('%', '').strip()
            maleRatioValue = int(maleRatioValue)
        except IndexError:
            maleRatioValue = ""
            pass
          
        except :
            maleRatioValue = ""
            pass
        # print(maleRatioValue)
        
        
        # 여자 선호 비율
        try:
        
          femaleRatio = soup2.select(".doughnut_label_wrap .label_info_value")[1]
        
          if femaleRatio:
            femaleRatioValue = femaleRatio.text.strip()
            femaleRatioValue = femaleRatioValue.replace('%','').strip()
            femaleRatioValue = int(femaleRatioValue)
          else:
            femaleRatioValueValue = ""
        except IndexError:
          femaleRatioValueValue = ""
          pass
        
        except:
          femaleRatioValueValue = ""
          pass
        # print(femaleRatioValue)
      
        
        # 상품 가격
        try:
          price = soup2.select('#product-right-top > div.product-detail__sc-w5wkld-0.jdBwFu > div.product-detail__sc-1p1ulhg-0.hauxDp > ul > li:nth-child(2) > div.product-detail__sc-1p1ulhg-6.bIRSux > span')

          priceText = [p.get_text(strip=True) for p in price]
          priceValue = None  # 초기값을 None으로 설정
          
          for p in priceText:
              p_values = p.replace('원', '').replace(',', '').strip().split('~')
              if p_values:
                  # 리스트에 값이 있으면 가장 마지막 값만 저장
                  priceValue = int(p_values[-1])
              else:
                priceValue = 0
        except:
          priceValue = 0
          pass
        # print(priceValue)
        
        # 제품 소재
        try:
          material = soup2.select('#page_product_detail > div.right_area.page_detail_product > div.product_info_table > table > tbody > tr:nth-child(1) > td > pre')
          
          if material:
            materialText = [m.get_text(strip=True) for m in material]
            materialValue = '  '.join(materialText)
          else:
            materialValue = ""
          
        except:
          materialValue = ""
          pass
      
        # print(materialValue)
        
        
        # 제품 색상
        try:
          colors = soup2.select('#page_product_detail > div.right_area.page_detail_product > div.product_info_table > table > tbody > tr:nth-child(2) > td > pre') 
        
          if colors:
            colorsText = [c.get_text(strip=True) for c in colors]
            colorsValue = '  '.join(colorsText)
          else:
            colorsValue = ""
        except:
          colorsValue = ""
          pass
           
        # print(colorsValue)
        
        mydict = {
          "상품명": titleValue,
          "종류": categoryValue,
          "분류": typesValue,
          "성별": genderValue,
          "조회수": viewsValue,
          "누적판매": salesValue,
          "선호연령" : agesValue,
          "선호연령비율": ageratiosValues,
          "선호성별" : sexValue,
          "남자선호비율" : maleRatio,
          "여자선호비율" : femaleRatio,
          "상품가격" : priceValue,
          "제품소재" : materialValue,
          "색상" : colorsValue
        }
        
        musinsaData.append(mydict)
        
        
    driver.quit()
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
                
    print("수집된 상품 갯수: ",len(musinsaData))

driver.quit()

수집된 상품 갯수:  90


In [10]:
copy = musinsaData.copy()

df = pd.DataFrame(copy)
excel_file_path = dt.datetime.now().strftime("MusinsaRank_%y%m%d_%H_%M.xlsx")
df.to_excel(excel_file_path, index=False)

print("fin :)")

fin :)
